In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import holidays
from scipy.fftpack import fft#푸리에 변환을 위한 코드입니다.
from scipy.stats import boxcox#박스콕스 변환을 위한 코드임
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
# ===== LightGBM 머신러닝 파이프라인 =====
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.express as px
import optuna
from optuna.samplers import TPESampler
from sklearn.linear_model import Ridge
from statsmodels.tsa.seasonal import STL
from sklearn.svm import SVR


from sklearn.svm import SVR

#기타
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_train = pd.read_csv("train_heat.csv")
df_test = pd.read_csv("test_heat.csv")
#열이름빼기
df_train.columns = df_train.columns.str.replace('train_heat.', '', regex=False)
#Unnamed:0제거
df_train = df_train.drop(columns=["Unnamed: 0"])
#test데이터 열이름 바꾸기
df_test.columns = [
    "tm", "branch_id", "ta", "wd", "ws",
    "rn_day", "rn_hr1", "hm", "si", "ta_chi","heat_demand"]


def create_time_series_features(df, target_cols=['ta', 'ws'], freq_hours=23):  # 24→23, hm→ws
    """시계열 분해를 통한 특성 생성 (STL 분해만 사용)"""
    print(f"🔄 STL 시계열 분해 특성 생성 중... (대상: {target_cols})")
    print("=" * 60)
    print("⚠️ heat_demand는 테스트 데이터에 없으므로 제외됩니다.")
    
    df_features = df.copy()
    
    # 브랜치별로 시계열 분해 수행
    for branch in tqdm(sorted(df['branch_id'].unique()), desc="브랜치별 STL 분해"):
        branch_mask = df_features['branch_id'] == branch
        branch_data = df_features[branch_mask].copy().sort_values('tm')
        
        if len(branch_data) < freq_hours * 7:  # 최소 7일 데이터 필요
            print(f"   ⚠️ 브랜치 {branch}: 데이터 부족 ({len(branch_data)}개) - 건너뜀")
            continue
        
        # 각 대상 변수별로 STL 분해 수행
        for col in target_cols:
            if col not in branch_data.columns:
                continue
                
            try:
                # 결측치 처리
                col_data = branch_data[col].interpolate().fillna(method='bfill').fillna(method='ffill')
                
                # STL 분해 (24시간 주기)
                try:
                    # 시간 인덱스 설정
                    ts_data = col_data.copy()
                    ts_data.index = pd.to_datetime(branch_data['tm'])
                    
                    # STL 분해
                    stl = STL(ts_data, seasonal=freq_hours, robust=True)
                    stl_result = stl.fit()
                    
                    # STL 결과 저장
                    indices = branch_data.index
                    df_features.loc[indices, f'{col}_stl_trend'] = stl_result.trend.values
                    df_features.loc[indices, f'{col}_stl_seasonal'] = stl_result.seasonal.values
                    df_features.loc[indices, f'{col}_stl_resid'] = stl_result.resid.values
                    
                    # 추가 파생 변수
                    df_features.loc[indices, f'{col}_detrend'] = col_data.values - stl_result.trend.values
                    df_features.loc[indices, f'{col}_seasonal_strength'] = np.abs(stl_result.seasonal.values)
                    
                    # 계절성 변동 지표
                    seasonal_std = np.std(stl_result.seasonal.values)
                    df_features.loc[indices, f'{col}_seasonal_volatility'] = seasonal_std
                    
                except Exception as e:
                    print(f"      ⚠️ STL 분해 실패 ({col}): {str(e)[:50]}")
                    continue
                
            except Exception as e:
                print(f"   ❌ 브랜치 {branch} {col} 분해 실패: {str(e)[:50]}")
                continue
    
    # 생성된 시계열 특성 목록
    time_series_features = [col for col in df_features.columns 
                           if any(pattern in col for pattern in ['_stl_', '_detrend', '_seasonal_strength', '_seasonal_volatility'])]
    
    print(f"\n✅ STL 시계열 분해 완료!")
    print(f"📊 생성된 시계열 특성: {len(time_series_features)}개")
    print(f"📋 특성 목록: {time_series_features[:10]}{'...' if len(time_series_features) > 10 else ''}")
    
    return df_features, time_series_features


def calculate_summer_apparent_temp(ta, hm):
    """여름철 체감온도 계산"""
    try:
        tw = ta * np.arctan(0.151977 * np.sqrt(hm + 8.313659)) \
             + np.arctan(ta + hm) \
             - np.arctan(hm - 1.676331) \
             + 0.00391838 * hm**1.5 * np.arctan(0.023101 * hm) \
             - 4.686035
        return -0.2442 + 0.55399 * tw + 0.45535 * ta - 0.0022 * tw**2 + 0.00278 * tw * ta + 3.0
    except:
        return np.nan

def calculate_winter_apparent_temp(ta, ws):
    """겨울철 체감온도 계산"""
    try:
        v = ws * 3.6  # m/s → km/h
        return 13.12 + 0.6215 * ta - 11.37 * v**0.16 + 0.3965 * ta * v**0.16
    except:
        return np.nan

def add_apparent_temp_features(df):
    df['month'] = df['tm'].dt.month
    df['apparent_temp'] = df.apply(lambda row:
        calculate_summer_apparent_temp(row['ta'], row['hm']) if 5 <= row['month'] <= 9
        else calculate_winter_apparent_temp(row['ta'], row['ws']),
        axis=1
    )
    return df


def branchwise_svr_impute(df, col, time_col='tm'):
    df = df.copy()
    # 시간 컬럼을 숫자형으로 변환 (timestamp, 초 단위)
    df['_time_num'] = pd.to_datetime(df[time_col]).astype(np.int64) // 10**9
    # branch별로 SVR 보간
    def impute_group(g):
        return svr_impute_series(g[col], g['_time_num'])
    # apply 결과를 원래 인덱스에 맞게 할당
    df[col] = df.groupby('branch_id', group_keys=False).apply(impute_group)
    df = df.drop(columns=['_time_num'])
    return df


def preprocess_weather_data(df):
    # 날짜 변환
    df['tm'] = pd.to_datetime(df['tm'], format='%Y%m%d%H')
    # 1. si: 08~18시가 아닐 때 -99는 0으로
    mask_outside_8_to_18 = (~df['tm'].dt.hour.between(8, 18)) & (df['si'] == -99)
    df.loc[mask_outside_8_to_18, 'si'] = 0

    # 2. wd에서 9.9는 NaN으로
    df['wd'] = df['wd'].replace(9.9, np.nan)

    # 3. -99 처리
    df.replace(-99, np.nan, inplace=True)


    # SVR 보간
    df = df.sort_values(['branch_id', 'tm'])

    numeric_cols = ['ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si', 'ta_chi', 'heat_demand']

    for branch in df['branch_id'].unique():
        print(f"   🏢 브랜치 {branch} SVR 보간 중...", end=" ")
        
        branch_mask = df['branch_id'] == branch
        branch_data = df[branch_mask].copy()

        # 시간 특성 생성
        branch_data['hour'] = branch_data['tm'].dt.hour
        branch_data['day_of_year'] = branch_data['tm'].dt.dayofyear
        branch_data['month'] = branch_data['tm'].dt.month

        for col in numeric_cols:
            if col in branch_data.columns:
                missing_mask = branch_data[col].isna()

                if missing_mask.sum() > 0:
                    train_mask = ~missing_mask

                    # 예측할 데이터 준비
                    X_train = branch_data.loc[train_mask, ['hour', 'day_of_year', 'month']].values
                    y_train = branch_data.loc[train_mask, col].values
                    X_pred = branch_data.loc[missing_mask, ['hour', 'day_of_year', 'month']].values

                    try:
                        scaler_X = StandardScaler()
                        scaler_y = StandardScaler()

                        X_train_scaled = scaler_X.fit_transform(X_train)
                        y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()

                        svr = SVR(kernel='rbf', C=1.0, gamma='scale')
                        svr.fit(X_train_scaled, y_train_scaled)

                        X_pred_scaled = scaler_X.transform(X_pred)
                        y_pred_scaled = svr.predict(X_pred_scaled)
                        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

                        # 보간 결과 반영
                        df.loc[branch_mask & missing_mask, col] = y_pred

                    except Exception as e:
                        print(f"\n   ⚠️ {col} SVR 실패 → 선형 보간 대체")
                        df.loc[branch_mask, col] = df.loc[branch_mask, col].interpolate(method='linear')

                # 남은 결측 ffill/bfill로 제거
                df.loc[branch_mask, col] = df.loc[branch_mask, col].fillna(method='ffill').fillna(method='bfill')

        print("✅")

    print("🎉 SVR 보간 완료 (조건 없이 전부 시도)")
    #보간후 음수나올 가능성존재
    df.loc[df['ta'] < 0, 'ta'] = 0
    df.loc[df['ws'] < 0, 'ws'] = 0

    # 📌 파생 변수 생성
    df['year'] = df['tm'].dt.year
    df['month'] = df['tm'].dt.month
    df['day'] = df['tm'].dt.day
    df['hour'] = df['tm'].dt.hour
    df['date'] = df['tm'].dt.date
    df['weekday'] = df['tm'].dt.weekday
    df['is_weekend'] = df['weekday'].isin([5,6]).astype(int)

    # 🇰🇷 한국 공휴일
    kr_holidays = holidays.KR()
    df['is_holiday'] = df['tm'].dt.date.apply(lambda x: int(x in kr_holidays))

    # 🕒 시간 지연
    for lag in [1, 2, 3]:
        df[f'ta_lag_{lag}'] = df.groupby('branch_id')['ta'].shift(lag)
        df[f'ta_lag_{lag}'] = df.groupby('branch_id')[f'ta_lag_{lag}'].transform(
        lambda x: x.fillna(method='bfill'))
    # 🔥 HDD / CDD
    df['HDD18'] = np.maximum(0, 18 - df['ta'])
    #df['CDD18'] = np.maximum(0, df['ta'] - 18)
    df['HDD20'] = np.maximum(0, 20 - df['ta'])
    #df['CDD20'] = np.maximum(0, df['ta'] - 20)

    df['ws_diff_6h'] = df.groupby('branch_id')['ws'].transform(lambda x: x.diff(6).bfill())
    df['ws_diff_12h'] = df.groupby('branch_id')['ws'].transform(lambda x: x.diff(12).bfill())
    df['ws_diff_24h'] = df.groupby('branch_id')['ws'].transform(lambda x: x.diff(24).bfill())



    #직접만든 체감온도
    df = add_apparent_temp_features(df)


    # 지점별 온도 편차
    branch_mean = df.groupby('branch_id')['ta'].transform('mean')
    df['branch_temp_abs_deviation'] = np.abs(df['ta'] - branch_mean)



    # 이동 평균 (3시간 단위 최대 24시간 = 8개)
    for n in [3, 6, 9, 12, 15, 18, 21, 24]:
        df[f'ta_3h_avg_{n}'] = df.groupby('branch_id')['ta'].transform(lambda x: x.rolling(n, min_periods=1).mean())

    # 불쾌지수
    df['DCI'] = 0.81 * df['ta'] + 0.01 * df['hm'] * (0.99 * df['ta'] - 14.3) + 46.3

    # 풍속 냉지수 (wchi)
    ws_kmh = df['ws'] * 3.6  # m/s -> km/h 변환
    df['wchi'] = 13.12 + 0.6215 * df['ta'] - 11.37 * ws_kmh**0.16 + 0.3965 * df['ta'] * ws_kmh**0.16

     # 풍속 고려 체감온도 (wind chill)
    df['wind_chill'] = 13.12 + 0.6215 * df['ta'] - 11.37 * df['ws']**0.16 + 0.3965 * df['ta'] * df['ws']**0.16

    # 실효온도
    df['e'] = (df['hm'] / 100) * 6.105 * np.exp((17.27 * df['ta']) / (237.7 + df['ta']))
    df['atemphi'] = df['ta'] + 0.33 * df['e'] - 0.70 * df['ws'] - 4.00

    # 주기성 인코딩
    df['dayofyear'] = df['tm'].dt.dayofyear
    df['dayofmonth'] = df['tm'].dt.day
    df['weekofyear'] = df['tm'].dt.isocalendar().week.astype(int)

    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dayofyear_sin'] = np.sin(2 * np.pi * df['dayofyear'] / 365)
    df['dayofyear_cos'] = np.cos(2 * np.pi * df['dayofyear'] / 365)
    df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)

    # 하루 5구간
    def time_slot(h): return int(h // 5)
    df['hour_slot_5'] = df['hour'].apply(time_slot)


    def compute_fft_feature(series, n=10):
        fft_vals = np.abs(fft(series.fillna(0)))
        s = pd.Series(fft_vals[:n], index=pd.Index([f'fft_{i}' for i in range(n)], name='fft_idx'))
        return s

    fft_cols = ['ta', 'hm', 'ws', 'ta_chi', 'apparent_temp']
    fft_features = []
    branch_ids = df['branch_id'].unique()
    fft_feature_dict = {bid: {} for bid in branch_ids}
    for col in fft_cols:
        if col not in df.columns:
            continue
        for branch_id in branch_ids:
            arr = df.loc[df['branch_id'] == branch_id, col].fillna(0).values
            fft_vals = np.abs(fft(arr))[:10]
            for i, val in enumerate(fft_vals):
                fft_feature_dict[branch_id][f'Nph_{col}_{i}'] = val
                
    # DataFrame으로 변환
    fft_features_df = pd.DataFrame.from_dict(fft_feature_dict, orient='index')
    # 원본 df와 merge
    df = df.merge(fft_features_df, left_on='branch_id', right_index=True, how='left')

    # 기온 차분
    df['ta_diff_6h'] = df.groupby('branch_id')['ta'].transform(lambda x: x.diff(6).bfill())
    df['ta_diff_12h'] = df.groupby('branch_id')['ta'].transform(lambda x: x.diff(12).bfill())
    df['ta_diff_24h'] = df.groupby('branch_id')['ta'].transform(lambda x: x.diff(24).bfill())


    # 일교차
    df['day_ta_max'] = df.groupby(['branch_id', df['tm'].dt.date])['ta'].transform('max')
    df['day_ta_min'] = df.groupby(['branch_id', df['tm'].dt.date])['ta'].transform('min')
    df['daily_range'] = df['day_ta_max'] - df['day_ta_min']

    # 일교차 변화량
    df['daily_range_shift'] = df.groupby('branch_id')['daily_range'].shift(1).bfill()
    df['daily_range_shift_ta'] = df['daily_range_shift']*df['ta']

    #ws변화량
    df['day_ws_max'] = df.groupby(['branch_id', df['tm'].dt.date])['ws'].transform('max')
    df['day_ws_min'] = df.groupby(['branch_id', df['tm'].dt.date])['ws'].transform('min')
    df['daily_range_ws'] = df['day_ws_max'] - df['day_ws_min']

    # 일교차 변화량
    df['daily_range_shift_ws'] = df.groupby('branch_id')['daily_range_ws'].shift(1).bfill()

    # 피크타임1
    df['peak_time1'] = 0
    df.loc[(df['hour'] >= 0) & (df['hour'] <= 6), 'peak_time1'] = 1
    df.loc[(df['hour'] > 6) & (df['hour'] <= 11), 'peak_time1'] = 2
    df.loc[(df['hour'] > 11) & (df['hour'] <= 18), 'peak_time1'] = 3
    df.loc[(df['hour'] > 18) & (df['hour'] <= 23), 'peak_time1'] = 4

    # 피크타임2
    df['peak_time2'] = 0
    df.loc[(df['hour'] >= 2) & (df['hour'] <= 10), 'peak_time2'] = 1




    # heating season
    df['heating_season'] = df['month'].isin([10,11,12,1, 2, 3,4]).astype(int)

    # 온도 범주화
    df['temp_category20'] = pd.cut(df['ta'], bins=[-np.inf, 20, np.inf], labels=['low', 'high'])
    df['temp_category18'] = pd.cut(df['ta'], bins=[-np.inf, 18, np.inf], labels=['low', 'high'])
    df['temp_category16'] = pd.cut(df['ta'], bins=[-np.inf, 16, np.inf], labels=['low', 'high'])

    # 오전/오후
    df['afternoon'] = (df['hour'] >= 12).astype(int)

    # 계절
    def get_season(month):
        return {
            12: 'winter', 1: 'winter', 2: 'winter',
            3: 'spring', 4: 'spring', 5: 'spring',
            6: 'summer', 7: 'summer', 8: 'summer',
            9: 'fall', 10: 'fall', 11: 'fall'
        }.get(month, 'unknown')
    df['season'] = df['month'].apply(get_season)

    # 한파 주의보/경보
    df['cold_watch'] = (df['ta'] <= -12).astype(int)  # 주의보
    df['cold_warning'] = (df['ta'] <= -15).astype(int)  # 경보
    # 시계열 분해 특성 생성 (heat_demand 제외)
    print("🚀 STL 시계열 분해 시작...")
    df, ts_features = create_time_series_features(df, target_cols=['ta', 'ws'])


    # 변환 대상 변수
    col = 'ta'
    '''
    df['ta_boxcox'] = np.nan
    df['ta_boxcox_lambda'] = np.nan
    df['ta_boxcox_shift'] = np.nan  # shift 값도 저장
    for branch, group in df.groupby('branch_id'):
        col = 'ta'
        min_val = group[col].min()
        if min_val <= 0:
            shift = abs(min_val) + 1e-4
        else:
            shift = 0
        shifted = group[col] + shift
        shifted = shifted.dropna()
        if shifted.nunique() > 1 and len(shifted) >= 2:
            transformed, fitted_lambda = boxcox(shifted)
            df.loc[shifted.index, 'ta_boxcox'] = transformed
            df.loc[shifted.index, 'ta_boxcox_lambda'] = fitted_lambda
            df.loc[shifted.index, 'ta_boxcox_shift'] = shift
        else:
            df.loc[group.index, 'ta_boxcox'] = np.nan
            df.loc[group.index, 'ta_boxcox_lambda'] = np.nan
            df.loc[group.index, 'ta_boxcox_shift'] = shift


    '''
    df = df.drop(columns=['date'])



    return df
#상호작용 처리못함
#군집화된 전처리 못함


#정규화 일단 min max +원핫인코딩
def scale_encode(df):
    cat_cols = [
        'peak_time1', 'peak_time2', 'heating_season','hour_slot_5',
        'temp_category16', 'temp_category18', 'temp_category20','afternoon', 'season','month','day','hour']

    # 범주형 변수 category화
    for col in cat_cols:
        if col in df.columns:
            df[col] = df[col].astype('category')

    # 원-핫 인코딩
    df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

    # 연속형 변수만 추출 (타겟, 날짜 등 제외)
    exclude_cols = ['heat_demand','peak_time1', 'peak_time2', 'heating_season','hour_slot_5',
        'temp_category16', 'temp_category18', 'temp_category20','afternoon', 'season','month','day','hour']
    num_cols = [col for col in df.columns
                if (df[col].dtype in [np.float64, np.int64]) and (col not in exclude_cols)]

    # MinMaxScaler 적용
    scaler = MinMaxScaler()
    df[num_cols] = scaler.fit_transform(df[num_cols])


    return df



df_train = preprocess_weather_data(df_train)
df_test = preprocess_weather_data(df_test)


df_train = scale_encode(df_train)
df_test = scale_encode(df_test)

df_train.to_csv('df_train_prescale.csv', index=True)
df_test.to_csv('df_test_prescale.csv', index=True)

   🏢 브랜치 A SVR 보간 중... ✅
   🏢 브랜치 B SVR 보간 중... ✅
   🏢 브랜치 C SVR 보간 중... ✅
   🏢 브랜치 D SVR 보간 중... ✅
   🏢 브랜치 E SVR 보간 중... ✅
   🏢 브랜치 F SVR 보간 중... ✅
   🏢 브랜치 G SVR 보간 중... ✅
   🏢 브랜치 H SVR 보간 중... ✅
   🏢 브랜치 I SVR 보간 중... ✅
   🏢 브랜치 J SVR 보간 중... ✅
   🏢 브랜치 K SVR 보간 중... ✅
   🏢 브랜치 L SVR 보간 중... ✅
   🏢 브랜치 M SVR 보간 중... ✅
   🏢 브랜치 N SVR 보간 중... ✅
   🏢 브랜치 O SVR 보간 중... ✅
   🏢 브랜치 P SVR 보간 중... ✅
   🏢 브랜치 Q SVR 보간 중... ✅
   🏢 브랜치 R SVR 보간 중... ✅
   🏢 브랜치 S SVR 보간 중... ✅
🎉 SVR 보간 완료 (조건 없이 전부 시도)
🚀 STL 시계열 분해 시작...
🔄 STL 시계열 분해 특성 생성 중... (대상: ['ta', 'ws'])
⚠️ heat_demand는 테스트 데이터에 없으므로 제외됩니다.


브랜치별 STL 분해: 100%|██████████| 19/19 [02:41<00:00,  8.49s/it]



✅ STL 시계열 분해 완료!
📊 생성된 시계열 특성: 12개
📋 특성 목록: ['ta_stl_trend', 'ta_stl_seasonal', 'ta_stl_resid', 'ta_detrend', 'ta_seasonal_strength', 'ta_seasonal_volatility', 'ws_stl_trend', 'ws_stl_seasonal', 'ws_stl_resid', 'ws_detrend']...
   🏢 브랜치 A SVR 보간 중... 
   ⚠️ heat_demand SVR 실패 → 선형 보간 대체
✅
   🏢 브랜치 B SVR 보간 중... 
   ⚠️ heat_demand SVR 실패 → 선형 보간 대체
✅
   🏢 브랜치 C SVR 보간 중... 
   ⚠️ heat_demand SVR 실패 → 선형 보간 대체
✅
   🏢 브랜치 D SVR 보간 중... 
   ⚠️ heat_demand SVR 실패 → 선형 보간 대체
✅
   🏢 브랜치 E SVR 보간 중... 
   ⚠️ heat_demand SVR 실패 → 선형 보간 대체
✅
   🏢 브랜치 F SVR 보간 중... 
   ⚠️ heat_demand SVR 실패 → 선형 보간 대체
✅
   🏢 브랜치 G SVR 보간 중... 
   ⚠️ heat_demand SVR 실패 → 선형 보간 대체
✅
   🏢 브랜치 H SVR 보간 중... 
   ⚠️ heat_demand SVR 실패 → 선형 보간 대체
✅
   🏢 브랜치 I SVR 보간 중... 
   ⚠️ heat_demand SVR 실패 → 선형 보간 대체
✅
   🏢 브랜치 J SVR 보간 중... 
   ⚠️ heat_demand SVR 실패 → 선형 보간 대체
✅
   🏢 브랜치 K SVR 보간 중... 
   ⚠️ heat_demand SVR 실패 → 선형 보간 대체
✅
   🏢 브랜치 L SVR 보간 중... 
   ⚠️ heat_demand SVR 실패 → 선형 보간 대체
✅
   🏢 브랜치 M SVR 보간 중... 
   

브랜치별 STL 분해: 100%|██████████| 19/19 [00:52<00:00,  2.77s/it]



✅ STL 시계열 분해 완료!
📊 생성된 시계열 특성: 12개
📋 특성 목록: ['ta_stl_trend', 'ta_stl_seasonal', 'ta_stl_resid', 'ta_detrend', 'ta_seasonal_strength', 'ta_seasonal_volatility', 'ws_stl_trend', 'ws_stl_seasonal', 'ws_stl_resid', 'ws_detrend']...


In [5]:
df_train = pd.read_csv('df_train_prescale.csv')
df_test = pd.read_csv('df_test_prescale.csv')
df_train = df_train.sort_values(['branch_id', 'tm'])
df_test = df_test.sort_values(['branch_id', 'tm'])



df_train = df_train.drop(columns=['year'])
df_train = df_train.drop(columns=['Unnamed: 0'])
df_test = df_test.drop(columns=['year'])
df_test = df_test.drop(columns=['Unnamed: 0'])
df_train = df_train.set_index('tm')
df_test = df_test.set_index('tm')
df_train = df_train.sort_index()
df_test = df_test.sort_index()

In [6]:
def run_3fold_pipeline_with_residual(df_train, df_test, target_col='heat_demand'):

    features = [col for col in df_train.columns if col != target_col]
    X = df_train[features]
    y = df_train[target_col]

    n = len(df_train)
    fold_size = n // 3

    val_rmses = []
    test_preds = []

    print(f"전체 데이터 길이: {n}, Fold 크기: {fold_size}\n")

    for fold in range(2):  # Fold 0, 1 수행 (3번째는 테스트용 데이터 분리)
        train_end = fold_size * (fold + 1)
        val_end = fold_size * (fold + 2)

        X_train = X.iloc[:train_end]
        y_train = y.iloc[:train_end]
        X_val = X.iloc[train_end:val_end]
        y_val = y.iloc[train_end:val_end]

        print(f"===== Fold {fold+1} =====")
        print(f"Train size: {len(X_train)}, Validation size: {len(X_val)}")

        # 1) LightGBM 베이지안 최적화 함수
        def lgb_objective(trial):
            params = {
                'objective': 'regression',
                'metric': 'rmse',
                'boosting_type': 'gbdt',
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 10, 300),
                'max_depth': trial.suggest_int('max_depth', 3, 15),
                'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 200),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
                'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
                'n_estimators': 1000,
                'random_state': 42,
                'n_jobs': -1
            }
            model = lgb.LGBMRegressor(**params)
            model.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
            )
            val_pred = model.predict(X_val)
            return np.sqrt(mean_squared_error(y_val, val_pred))

        lgb_study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
        lgb_study.optimize(lgb_objective, n_trials=20, show_progress_bar=True)

        best_lgb_params = lgb_study.best_params
        best_lgb_params.update({
            'objective': 'huber',
            'metric': 'rmse',
            'boosting_type': 'gbdt',
            'n_estimators': 1000,
            'random_state': 42,
            'n_jobs': -1
        })

        # 2) LightGBM 최적 모델 학습
        lgb_model = lgb.LGBMRegressor(**best_lgb_params)
        lgb_model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
        )

        val_pred_lgb = lgb_model.predict(X_val)
        val_rmse_lgb = np.sqrt(mean_squared_error(y_val, val_pred_lgb))
        print(f"LightGBM Fold {fold+1} Validation RMSE: {val_rmse_lgb:.4f}")

        # 3) 잔차 계산
        residual_train = y_train - lgb_model.predict(X_train)
        residual_val = y_val - val_pred_lgb

        # 4) XGBoost 베이지안 최적화 함수 (fold별로 동일하게 분할)
        def xgb_objective(trial):
            params = {
                'objective': 'reg:pseudohubererror',
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'max_depth': trial.suggest_int('max_depth', 2, 10),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
                'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
                'random_state': 42,
                'n_jobs': -1
            }
            model = xgb.XGBRegressor(**params)
            model.fit(
                X_train, residual_train,
                eval_set=[(X_val, residual_val)],
                verbose=0
            )
            val_pred = model.predict(X_val)
            return np.sqrt(mean_squared_error(residual_val, val_pred))

        xgb_study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
        xgb_study.optimize(xgb_objective, n_trials=20, show_progress_bar=True)

        best_xgb_params = xgb_study.best_params
        best_xgb_params.update({
            'objective': 'reg:pseudohubererror',
            'random_state': 42,
            'n_jobs': -1
        })

        # 5) XGBoost 최적 모델 학습 (잔차 예측용, fold별로)
        xgb_model = xgb.XGBRegressor(**best_xgb_params)
        xgb_model.fit(
            X_train, residual_train,
            eval_set=[(X_val, residual_val)],
            verbose=100
        )

        val_pred_residual = xgb_model.predict(X_val)
        val_pred_final = val_pred_lgb + val_pred_residual
        val_rmse_final = np.sqrt(mean_squared_error(y_val, val_pred_final))
        print(f"Residual Stacking Fold {fold+1} Validation RMSE: {val_rmse_final:.4f}")

        val_rmses.append(val_rmse_final)

        # fold별로 test 예측 저장
        test_pred_lgb = lgb_model.predict(df_test[features])
        test_pred_residual = xgb_model.predict(df_test[features])
        test_pred_fold = test_pred_lgb + test_pred_residual
        test_preds.append(test_pred_fold)

        print("-----------------------------")

    avg_val_rmse = np.mean(val_rmses)
    avg_test_pred = np.mean(test_preds, axis=0)

    print(f"\n최종 평균 Validation RMSE: {avg_val_rmse:.4f}")

    df_test[target_col] = avg_test_pred

    return {
        'val_rmse': avg_val_rmse,
        'val_rmses': val_rmses,
        'test_pred': avg_test_pred,
        'test_index': df_test.index
    }

In [ ]:
branch_rmse_results = {}
test_pred_dict = {}

branch_ids = df_train['branch_id'].unique()

for branch in branch_ids:
    train_branch = df_train[df_train['branch_id'] == branch].copy()
    test_branch = df_test[df_test['branch_id'] == branch].copy()
    
    # branch_id는 모델에 불필요하면 제거
    train_branch = train_branch.drop(columns=['branch_id'])
    test_branch = test_branch.drop(columns=['branch_id'])
    
    target_col = 'heat_demand'
    
    results = run_3fold_pipeline_with_residual(train_branch, test_branch, target_col)
    
    branch_rmse_results[branch] = {
        'val_rmse': results['val_rmse']
    }
    
    test_pred_dict[branch] = pd.DataFrame({
        'branch_ID': branch,
        'TM': results['test_index'],
        'heat_demand': results['test_pred']
    }).set_index('TM')


# test_pred_dict의 모든 branch 결과를 하나의 DataFrame으로 병합
merged_df = pd.concat(test_pred_dict.values()).reset_index()


# CSV 파일로 저장
merged_df.to_csv('250464_test.csv', index=False)  


In [8]:
df1 = pd.read_csv('250464_test.csv')
df1['TM'] = pd.to_datetime(df1['TM'])
df1 = df1.sort_values(by=['branch_ID', 'TM']).reset_index(drop=True)
df2 = pd.read_csv('test_heat.csv')
df2['heat_demand'] = df1['heat_demand'].round(1)
df2.to_csv('250464.csv', index=False) 

In [9]:
#huber지표랑 교차검증 순차로하는식으로 3fold로해서 연도별로 짜름 까지 고려